# Implementing a LSTM layer

## Introduction

The objective of this exercise is get an in-depth understanding of the LSTM layer as it is a crucial layer in Recurrent Neural Networks. To achieve this, you will simply implement your own LSTM layer ``MyLSTM`` and make sure that it yields the same results as [nn.LSTM](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html?highlight=lstm#torch.nn.LSTM).

## Contents:

1. Utils
2. Reading txt files, building a vocabulary and creating a dataset
2. Implementing a custom layer in Pytorch: ``MyLSTM``
3. Using ``MyLSTM`` inside a neural network model
4. Testing ``MyLSTM`` by comparing it to [nn.LSTM](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html?highlight=lstm#torch.nn.LSTM)

In [1]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from datetime import datetime
from torch.utils.data import DataLoader, TensorDataset

import numpy as np
import torchtext
import os
import re
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

torch.manual_seed(265)
# We use torch.double to get the same results as Pytorch
torch.set_default_dtype(torch.double)

## 1. Utils

Nothing to see in the cell below, just the definition functions we'll need later, you don't even need to read them, just know that there are 2 functions:

- ``train``: Train a model, save weight values of the LSTM layer for each epoch of the training. 
- ``relative_error(a, b)``: Compute the relative error of ``b``

In [2]:
device = torch.device('cpu')
print(f"Device {device}.")

def train(n_epochs, optimizer, model, loss_fn, train_loader):
    """
    Train our model and save weight values
    """
    n_batch = len(train_loader)
    losses_train = []
    model.train()
    optimizer.zero_grad(set_to_none=True)
    
    weight = {
        "weight_ih_l0" : [],
        "weight_hh_l0" : [],
        "bias_ih_l0" : [],
        "bias_hh_l0" : [],
    }
    
    for epoch in range(1, n_epochs + 1):
        
        loss_train = 0.0
        for contexts, labels in train_loader:

            contexts = torch.permute(contexts, (1,0,2))
            # We use torch.double to get the same results as Pytorch
            contexts = contexts.to(device=device, dtype=torch.double) 
            
            labels = labels.to(device=device, dtype=torch.double)

            outputs = model(contexts).squeeze()
            
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            loss_train += loss.item()
            
        losses_train.append(loss_train / n_batch)
        
        # Here we store weight values at each step of the training process
        # The "0" written after each layer name refers to the fact that this is the first LSTM layer (and only one)
        with torch.no_grad():
            weight["weight_ih_l0"].append(model.lstm.weight_ih_l0.data.clone().detach())
            weight["weight_hh_l0"].append(model.lstm.weight_hh_l0.data.clone().detach())
            weight["bias_ih_l0"].append(model.lstm.bias_ih_l0.data.clone().detach())
            weight["bias_hh_l0"].append(model.lstm.bias_hh_l0.data.clone().detach())

        print('{}  |  Epoch {}  |  Training loss {:.5f}'.format(
            datetime.now().time(), epoch, loss_train / n_batch))
    return weight


def relative_error(a, b):
    return (torch.norm(a - b) / torch.norm(a))

Device cpu.


## 2. Reading txt files, building a vocabulary and creating a dataset

The objective this week is only to implement a LSTM layer. We will then use a short text and keep only a very short vocabulary and simply try to predict if, given a sequence, the next word will be a known word. (`<unk>` token or not)

There is nothing to do in this cell, apart from running it. You might want to read it carefully though, as it is an important step of the pipeline in pytorch when dealing with text data. Note that last week there were also a tutorial on text data (see `06 - Text data`).

In [3]:
# tokenizer will split a long text into a list of english words
TOKENIZER_EN = get_tokenizer('basic_english')
# Minimum number of occurence of a word in the text to add it to the vocabulary
MIN_FREQ = 2

def read_files(datapath='./data_train/'):
    """
    Return a list of strings, one for each line in each .txt files in 'datapath'
    """
    # Find all txt files in directory 
    files = os.listdir(datapath)
    files = [datapath + f for f in files if f.endswith(".txt")]
    
    # Stores each line of each book in a list
    lines = []
    for f_name in files:
        with open(f_name) as f:
            lines += f.readlines()
    return lines

def tokenize(lines, tokenizer=TOKENIZER_EN):
    """
    Tokenize the list of lines
    """
    list_text = []
    for line in lines:
        list_text += tokenizer(line)
    return list_text

def yield_tokens(lines, tokenizer=TOKENIZER_EN):
    """
    Yield tokens, ignoring names and digits to build vocabulary
    """
    # Match any word containing digit
    no_digits = '\w*[0-9]+\w*'
    # Match word containing a uppercase 
    no_names = '\w*[A-Z]+\w*'
    # Match any sequence containing more than one space
    no_spaces = '\s+'
    
    for line in lines:
        line = re.sub(no_digits, ' ', line)
        line = re.sub(no_names, ' ', line)
        line = re.sub(no_spaces, ' ', line)
        yield tokenizer(line)

def count_freqs(words, vocab):
    """
    Count occurrences of each word in vocabulary in the data
    
    Useful to get some insight on the data and to compute loss weights
    """
    freqs = torch.zeros(len(vocab), dtype=torch.int)
    for w in words:
        freqs[vocab[w]] += 1
    return freqs

def create_vocabulary(lines, min_freq=MIN_FREQ):
    """
    Create a vocabulary (list of known tokens) from a list of strings
    """
    # vocab contains the vocabulary found in the data, associating an index to each word
    vocab = build_vocab_from_iterator(yield_tokens(lines), min_freq=min_freq, specials=["<unk>"])
    # Since we removed all words with an uppercase when building the vocabulary, we skipped the word "I"
    vocab.append_token("i")
    # Value of default index. This index will be returned when OOV (Out Of Vocabulary) token is queried.
    vocab.set_default_index(vocab["<unk>"])
    return vocab

In [4]:
# ----------------------- Tokenize texts -------------------------------
# Get lists of strings, one for each line in each .txt files in 'datapath' 
lines_books_train = read_files('./data_train/')

# List of words contained in the dataset
words_train = tokenize(lines_books_train)

# ----------------------- Create vocabulary ----------------------------
# Create vocabulary based on the words in the training dataset
vocab = create_vocabulary(lines_books_train, min_freq=MIN_FREQ)

# ------------------------ Quick analysis ------------------------------
VOCAB_SIZE = len(vocab)
print("Total number of words in the training dataset:     ", len(words_train))
print("Number of distinct words in the training dataset:  ", len(set(words_train)))
print("Number of distinct words kept (vocabulary size):   ", VOCAB_SIZE)

freqs = count_freqs(words_train, vocab)
print("occurences:\n", [(f.item(), w) for (f, w)  in zip(freqs, vocab.lookup_tokens(range(VOCAB_SIZE)))])

Total number of words in the training dataset:      240
Number of distinct words in the training dataset:   98
Number of distinct words kept (vocabulary size):    32
occurences:
 [(78, '<unk>'), (18, 'a'), (12, 'down'), (12, 'go'), (12, 'medicine'), (19, 'the'), (10, 'of'), (6, 'helps'), (6, 'sugar'), (5, 'to'), (6, 'spoonful'), (4, 'that'), (3, '!'), (6, 'and'), (3, 'delightful'), (3, 'every'), (3, 'find'), (3, 'his'), (3, 'job'), (3, 'most'), (3, 'way'), (2, '('), (2, ')'), (2, 'fun'), (2, 'hence'), (2, 'is'), (2, 'little'), (2, 's'), (2, 'task'), (4, 'they'), (2, 'very'), (0, 'i')]


Create a ``(contexts, targets)`` toydataset such that for each ``(c, t)`` context/target pair in the dataset, ``t = 0`` if the next word after the sequence ``c`` is the ``<unk>`` token (i.e. the first element of the vocabulary representing words that are outside the vocabulary), and ``t = 1``.

In [5]:
# ------------------------ Define targets ------------------------------
def compute_label(w):
    """
    helper function to define MAP_TARGET
    
    - 0 = 'unknown word'
    - 1 = 'is an actual word'
    """
    if w in ['<unk>']:
        return 0
    else:
        return 1

# true labels for this task:
MAP_TARGET = {
    vocab[w]:compute_label(w) for w in vocab.lookup_tokens(range(VOCAB_SIZE))
}

# context size for this task 
CONTEXT_SIZE = 3


# ---------------- Define context / target pairs -----------------------
def create_dataset(
    text, vocab, 
    context_size=CONTEXT_SIZE, map_target=MAP_TARGET
):
    """
    Create a pytorch dataset of context / target pairs from a text
    """
    
    n_text = len(text)
    n_vocab = len(vocab)
    
    # Change labels if only a few target are kept, otherwise, each word is
    # associated with its index in the vocabulary
    if map_target is None:
        map_target = {i:i for i in range(n_vocab)}
    
    # Transform the text as a list of integers.
    txt = [vocab[w] for w in text]

    # Start constructing the context / target pairs...
    contexts = []
    targets = []
    for i in range(n_text - context_size):
        
        # Word used to define target
        t = txt[i + context_size]
        
        # Context before the target
        c = txt[i:i + context_size]
        
        targets.append(map_target[t])
        # Normally we should use word embedding, and not hot encoding, but we 
        # skip that part for this exercise
        contexts.append(F.one_hot(torch.tensor(c), num_classes=n_vocab))
            
    contexts = torch.stack(contexts)
    targets = torch.tensor(targets)
    # Create a pytorch dataset out of these context / target pairs
    return TensorDataset(contexts, targets)

data = create_dataset(words_train, vocab)

## 2. Implementing a LSTM layer in pytorch

### Illustration and notations of LSTM layers in pytorch 

![LSTM in pytorch](lstm.png) ![LSTM in pytorch](nn.lstm.png)

image credits [diagram](https://stackoverflow.com/questions/48302810/whats-the-difference-between-hidden-and-output-in-pytorch-lstm/48305882#48305882), [nn.LSTM](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html?highlight=lstm#torch.nn.LSTM)

Reminder about Pytorch notations and dimensions:

- ``N``: batch size
- ``L``: sequence length (number of words given as input)
- $H_{in}$: input size (dimension of a word)
- $H_{cell}, H_{out}$: hidden size (dimension of tensor between cells) 
- $h_t^{(l)}$​ is the hidden state at time $t$ in layer $l$
- $c_t^{(l)}$​ is the cell state at time $t$ in layer $l$
- $x_t^{(l)}$​ is the input at time $t$ in layer $l$
- $h_{t−1}^{(l)}$​ is the hidden state of the layer at time $t-1$ or the initial hidden state at time $0$ in layer $l$

Note that for a **many-to-one LSTM** network (i.e network with input sequence of length $L>1$ and output sequence of length $L=1$), we are only interested in $h_n^{(w)}$, i.e the final hidden state of the last layer.


### Specific settings for this exercise

We will restrict the scope of this exercise by setting the following parameters of the [nn.LSTM](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html?highlight=lstm#torch.nn.LSTM) layer. (those are the defaults ones)

- ``num_layers = 1``
- ``bias = True``
- ``batch_first = False``
- ``dropout = 0``
- ``bidirectional = False``
- ``proj_size = 0``

In these specific settings:

- we give an input ``input`` of shape $(L, N, H_{in})$
- we "only" have to compute: 
    - ``LSTM.weight_ih_l0[k]``: the learnable input-hidden weights  $(W_{ii}|W_{if}|W_{ig}|W_{io})$ of the $k^{th}$ layer. For $k=0$ (which is in fact our unique case here, since we have only one layer) the shape is ``(4*hidden_size, input_size)``. Otherwise, the shape is ``(4*hidden_size, hidden_size)``
    - ``LSTM.weight_hh_l0[0]``: the learnable hidden-hidden weights $(W_{hi}|W_{hf}|W_{hg}|W_{ho})$, all of shape ``(4*hidden_size, hidden_size)``.
    - ``LSTM.bias_ih_l0[0]``  : the learnable input-hidden bias of the layer $(b_{ii}|b_{if}|b_{ig}|b_{io})$, all of shape ``(4*hidden_size)``.
    - ``LSTM.bias_hh_l0[0]``  : the learnable hidden-hidden bias of the layer $(b_{hi}|b_{hf}|b_{hg}|b_{ho})$, all of shape ``(4*hidden_size)``.
- we get as output ``output, (h_n, c_n)`` such that:
    - ``output`` has shape ``(L, N, H_out​)``, containing the output features ``h_t`` from the last layer of the LSTM, for each $t$
    - ``h_n`` has shape ``(num_layers, N, H_out​)`` (with ``num_layers = 1``) containing the final hidden state for each layer.
    - ``c_n`` has shape ``(num_layers, N, H_cell​)`` (with ``num_layers = 1``) containing the final cell state for each layer.
    

### TODO

Define a LSTM module, by implementing the operations described in [nn.LSTM](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html?highlight=lstm#torch.nn.LSTM). 

**WARNING** Pytorch doesn't like to compute the backward pass on recurrent operations! When implementing the instructions for each cell, don't hesitate to add '.clone' whenever you use an old value of a given tensor so that Pytorch creates a new component inside the computational graph.

In [6]:
class MyLSTM(nn.Module):
    
    def __init__(
        self, 
        input_size:int,
        hidden_size:int,
    ):
        super().__init__()
        self.input_size = int(input_size)
        self.hidden_size = int(hidden_size)

        self.weight_ih_l0 = nn.Parameter(torch.Tensor( (4*self.hidden_size, self.input_size) ))
        self.weight_hh_l0 = nn.Parameter(torch.Tensor( (4*self.hidden_size, self.hidden_size) )) 
        self.bias_ih_l0 = nn.Parameter(torch.Tensor( (4*self.hidden_size) )) 
        self.bias_hh_l0 = nn.Parameter(torch.Tensor( (4*self.hidden_size) )) 

        
    def forward(self, x):
        (L, N, H_in) = x.shape

        output = torch.zeros((L, N, self.hidden_size))
        h = torch.zeros((L+1, N, self.hidden_size))
        c = torch.zeros((L+1, N, self.hidden_size))
        
        s = [self.hidden_size*k for k in range(4)]
        e = [self.hidden_size*k for k in range(1,5)]
        
        # shorthand
        mm = torch.matmul
        for j in range(N):
            for t in range(L):
                i = torch.sigmoid( mm(self.weight_ih_l0[s[0]:e[0]], x[t, j]) + self.bias_ih_l0[s[0]:e[0]] + mm(self.weight_hh_l0[s[0]:e[0]], h[t, j].clone()) + self.bias_hh_l0[s[0]:e[0]])
                f = torch.sigmoid( mm(self.weight_ih_l0[s[1]:e[1]], x[t, j]) + self.bias_ih_l0[s[1]:e[1]] + mm(self.weight_hh_l0[s[1]:e[1]], h[t, j].clone()) + self.bias_hh_l0[s[1]:e[1]])
                g = torch.tanh( mm(self.weight_ih_l0[s[2]:e[2]], x[t, j]) + self.bias_ih_l0[s[2]:e[2]] +    mm(self.weight_hh_l0[s[2]:e[2]], h[t, j].clone()) + self.bias_hh_l0[s[2]:e[2]])
                o = torch.sigmoid( mm(self.weight_ih_l0[s[3]:e[3]], x[t, j]) + self.bias_ih_l0[s[3]:e[3]] + mm(self.weight_hh_l0[s[3]:e[3]], h[t, j].clone()) + self.bias_hh_l0[s[3]:e[3]])
                c[t+1, j] = f.clone() * c[t, j].clone() + i.clone() * g.clone()
                h[t+1, j] = o.clone() * torch.tanh(c[t+1, j].clone())
                output[t, j] = h[0, j]

        return output, (h, c)

## 3. Using MyLSTM inside a neural network model

Here is defined a very basic neural network to test ``MyLSTM``. It consists of: 

- a LSTM layer (``MyLSTM`` or ``nn.LSTM``) (see ``lstm_type`` parameter)
- a fully connected layer with 1 output

You don't have to do anything here, simply run the cell

In [7]:
class MyNet(nn.Module):
    """
    Simple net with only one LSTM layer and one fc layer. 
    
    The LSTM layer can be ``MyLSTM`` or ``nn.LSTM``
    """

    def __init__(
        self,
        input_size:int,
        hidden_size:int,
        lstm_type:str = 'custom',
    ):
        
        super().__init__()
        # Use MyLSTM
        if lstm_type == 'custom':
            self.lstm = MyLSTM(input_size, hidden_size)
        # or use pytorch's LSTM
        else:
            self.lstm = nn.LSTM(input_size, hidden_size)
        self.fc1 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # LSTM outputs: (out, (h, c)) with h of shape (num_layer, N, H_out) and we want h[-1,:,:]
        output, (h_n, c_n) = self.lstm(x)
        out = self.fc1(h_n[-1])
        return out

## 4. Testing MyLSTM by comparing it to nn.LSTM

MyLSTM and nn.LSTM are compared by storing after each epoch:

- Weight and biases values of the LSTM layer
- Training loss

**NOTE**

- Your implementation will probably be much slower than nn.LSTM.
- If your implementation is correct, the training loss printed should be identical and the expected relative error should be around 1e-16.

In [8]:
# DONT USE GPU!!! IT WOULD REQUIRE USING register_buffer TO MOVE THE MODEL CORRECTLY TO
# THE GPU. (See https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.register_buffer)
# And you probably don't want to spend the entire week learning how to use this properly
device = torch.device('cpu')
print(f"Training on device {device}.")



# We set shuffle to False so that we can compare both models more accurately
train_loader = DataLoader(data, batch_size=64, shuffle=False)
loss_fn = nn.BCEWithLogitsLoss()

# These parameters don't matter much in this assignment
n_epochs = 4    
lr = 0.1
hidden_size = 12

torch.manual_seed(265)
# Using MyLSTM
model01 = MyNet(VOCAB_SIZE, hidden_size, lstm_type='custom').to(device=device) 
# Using nn.LSTM

torch.manual_seed(265)
model02 = MyNet(VOCAB_SIZE, hidden_size, lstm_type='pytorch').to(device=device) 

# Make both models start with the same weight values
with torch.no_grad():
    # The "0" written after each layer name refers to the fact that this is the first LSTM layer (and only one)
    model01.lstm.weight_ih_l0.data = model02.lstm.weight_ih_l0.data.clone()
    model01.lstm.weight_hh_l0.data = model02.lstm.weight_hh_l0.data.clone()
    model01.lstm.bias_ih_l0.data = model02.lstm.bias_ih_l0.data.clone()
    model01.lstm.bias_hh_l0.data = model02.lstm.bias_hh_l0.data.clone()
    model01.fc1.bias.data = model02.fc1.bias.data.clone()
    model01.fc1.weight.data = model02.fc1.weight.data.clone()

print("\n ========= Training using MyLSTM =========")
optimizer = optim.SGD(model01.parameters(), lr=lr)
weight01 = train(
    n_epochs = n_epochs,
    optimizer = optimizer, 
    model = model01,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

print("\n ========= Training using nn.LSTM =========")


optimizer = optim.SGD(model02.parameters(), lr=lr)
weight02= train(
    n_epochs = n_epochs,
    optimizer = optimizer, 
    model = model02,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

print("\n ======= Relative error:     nn.LSTM    VS    MyLSTM   =========")
for (k1, v1), (k2, v2) in zip(weight01.items(), weight02.items()):
    print('-'*20, k1, '-'*20)
    print([relative_error(v1[i],v2[i]) for i in range(len(v1))] )

Training on device cpu.

 ========= Training using MyLSTM =========
21:53:07.802600  |  Epoch 1  |  Training loss 0.65921
21:53:08.687727  |  Epoch 2  |  Training loss 0.64978
21:53:09.517562  |  Epoch 3  |  Training loss 0.64260
21:53:10.331952  |  Epoch 4  |  Training loss 0.63711

 ========= Training using nn.LSTM =========
21:53:10.346035  |  Epoch 1  |  Training loss 0.65921
21:53:10.352859  |  Epoch 2  |  Training loss 0.64978
21:53:10.359853  |  Epoch 3  |  Training loss 0.64260
21:53:10.366550  |  Epoch 4  |  Training loss 0.63711

 ======= Relative error:     nn.LSTM    VS    MyLSTM   =========
-------------------- weight_ih_l0 --------------------
[tensor(0.), tensor(1.3273e-19), tensor(4.2825e-18), tensor(4.3154e-18)]
-------------------- weight_hh_l0 --------------------
[tensor(4.1897e-18), tensor(8.2476e-18), tensor(8.1194e-18), tensor(1.0694e-17)]
-------------------- bias_ih_l0 --------------------
[tensor(0.), tensor(1.7132e-17), tensor(2.5687e-17), tensor(2.5676e-17)]